# Github Natural Language Processing

#### by Ryan McCall, David Wederstrandt Sr., and Noah Melngailis

## Introduction

The purpose of this project was to see if we could use the README text of GitHub repos to predict the programming language that is used to program the majority of the repository.

We did this by using the most starred repos for JavaScript, Python, Java, and PHP as they are the top four languages on GitHub. We scraped the names of the top 200 repos for each of these languages for a total of 800 name. We then used the GitHub API to bring in the repos main language and readme contents.

We then took these files and reduced them down to keywords using text normalization with string methods, unicode conversion, regex substitution, word stemming, and word lemmatization.

Next we explored the documents by looking at similarities and differences between them. We looked at the common words shared between the documents and the common words shared by programming languages. We looked at the importance of words in each of the documents and the importance of the words based on the language of the repository. We also looked at the difference in the amount of words used by each of the these languages.

We then went on to create a model that used a Decision Tree Classifier to predict the programming language of the repository. We found that the single words used in each README yielded the most accurate model which was able to predict the language of the test set with 75% accuracy.

## Project

### Imports

In [16]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report
from sklearn import tree

import graphviz
from graphviz import Graph

from prepare import wrangle_data

### Acquire

For the acquisition of the data, our acquire.py file starts by scrapping the names of the 200 most starred repos for that had JavaScript, Python, Java, or PHP as their main language. This gave us a list of 800 names that we stored in a .csv file. That was read in by our acquire file and used to make request for the name, main programming language, and README contents of the repositories. This information is then stored in a .json file that can be read into a pandas DataFrame.

This is all achieved by running `python acquire.py` in the terminal which creates the repo.csv (repo names) and data.json (repo info) files.

In [3]:
data = pd.read_json('data.json')
data

,repo,language,readme_contents
0,freeCodeCamp/freeCodeCamp,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...
1,vuejs/vue,JavaScript,"<p align=""center""><a href=""https://vuejs.org"" ..."
2,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot..."
3,airbnb/javascript,JavaScript,# Airbnb JavaScript Style Guide() {\n\n*A most...
4,d3/d3,JavaScript,"# D3: Data-Driven Documents\n\n<a href=""https:..."
...,...,...,...
795,isohuntto/openbay,PHP,Openbay is not supported\n=======\n\nWarm gree...
796,geocoder-php/Geocoder,PHP,Geocoder\n========\n\n[![Build Status](https:/...
797,maximebf/php-debugbar,PHP,# PHP Debug Bar\n\n[![Latest Stable Version](h...
798,nunomaduro/phpinsights,PHP,"<p align=""center"">\n <img src=""https://raw.gi..."


### Prepare

Now this is what our some of our readme contents look like:

In [13]:
data.readme_contents[1]

'<p align="center"><a href="https://vuejs.org" target="_blank" rel="noopener noreferrer"><img width="100" src="https://vuejs.org/images/logo.png" alt="Vue logo"></a></p>\n\n<p align="center">\n  <a href="https://circleci.com/gh/vuejs/vue/tree/dev"><img src="https://img.shields.io/circleci/project/github/vuejs/vue/dev.svg?sanitize=true" alt="Build Status"></a>\n  <a href="https://codecov.io/github/vuejs/vue?branch=dev"><img src="https://img.shields.io/codecov/c/github/vuejs/vue/dev.svg?sanitize=true" alt="Coverage Status"></a>\n  <a href="https://npmcharts.com/compare/vue?minimal=true"><img src="https://img.shields.io/npm/dm/vue.svg?sanitize=true" alt="Downloads"></a>\n  <a href="https://www.npmjs.com/package/vue"><img src="https://img.shields.io/npm/v/vue.svg?sanitize=true" alt="Version"></a>\n  <a href="https://www.npmjs.com/package/vue"><img src="https://img.shields.io/npm/l/vue.svg?sanitize=true" alt="License"></a>\n  <a href="https://chat.vuejs.org/"><img src="https://img.shields.i

---

#### As you may see, there is HTML and Markdown text scattered throught the text that we want to capture. After several cleaning steps the above text was able to be turned into this:

In [14]:
df = wrangle_data()
df.lemmatized[1]

"supporting vuejsvuejs mitlicensed open source project ongoing development made possible entirely support awesome backer ' like join please consider become backer sponsor patreon become backer sponsor open collective onetime donation via paypal cryptocurrencies ' difference patreon opencollectivefunds donated via patreon go directly support evan ' fulltime work vuejs fund donated via opencollective managed transparent expense used compensating work expense core team member sponsoring community event namelogo receive proper recognition exposure donating either platformspecial sponsorsplatinum sponsorsplatinum sponsor chinagold sponsorssponsors via open collectiveplatinumgold introductionvue pronounced vju like view progressive framework building user interface designed ground incrementally adoptable easily scale library framework depending different use case consists approachable core library focus view layer ecosystem supporting library help tackle complexity large singlepage applicati

The steps to achieve this are as follows:

* First, all html tags (anything between a '<' and '>') were removed.
* Next, all urls were removed.
* Newline characters and extra spaces were also removed.
* Then anything that wasn't alphanumeric was removed.
* Next, stopwords (a, an, the, etc.) were removed.
* Finally the text was stemmed and lemmatized, each of which was placed in it's own column

In [15]:
df

,repo,language,readme_contents,stemmed,lemmatized
0,freeCodeCamp/freeCodeCamp,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...,freecodecamporg social bannerpul request welco...,freecodecamporg social bannerpull request welc...
1,vuejs/vue,JavaScript,"<p align=""center""><a href=""https://vuejs.org"" ...",support vuejsvuej mitlicens open sourc project...,supporting vuejsvuejs mitlicensed open source ...
2,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",bootstrapsleek intuit power frontend framework...,bootstrapsleek intuitive powerful frontend fra...
3,airbnb/javascript,JavaScript,# Airbnb JavaScript Style Guide() {\n\n*A most...,airbnb javascript style guid mostli reason app...,airbnb javascript style guide mostly reasonabl...
4,d3/d3,JavaScript,"# D3: Data-Driven Documents\n\n<a href=""https:...",d3 datadriven documentsd3 d3j javascript libra...,d3 datadriven documentsd3 d3js javascript libr...
...,...,...,...,...,...
795,isohuntto/openbay,PHP,Openbay is not supported\n=======\n\nWarm gree...,openbay supportedwarm greet guestsisohunt team...,openbay supportedwarm greeting guestsisohunt t...
796,geocoder-php/Geocoder,PHP,Geocoder\n========\n\n[![Build Status](https:/...,geocoderbuild statussoftwar licenselicens impo...,geocoderbuild statussoftware licenselicense im...
797,maximebf/php-debugbar,PHP,# PHP Debug Bar\n\n[![Latest Stable Version](h...,php debug barlatest stabl version total downlo...,php debug barlatest stable version total downl...
798,nunomaduro/phpinsights,PHP,"<p align=""center"">\n <img src=""https://raw.gi...",full document visit phpinsightscomphp insight ...,full documentation visit phpinsightscomphp ins...


### Exploration

### Modeling

#### Bag of Words Model

First we started with a model that uses how often each unique word from all documents appears in each document, also know as a bag of words. We will use the CountVectorizer to create a sparse matrix containing this information. We then split the data and feed the training set into a Decision Tree Classifier with a max depth of 10, which was determined through tuning on the validation set.

In [33]:
cv = CountVectorizer()
X = cv.fit_transform(df.lemmatized)
y = df.language

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

tree = DecisionTreeClassifier(max_depth=10, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score: 83.12%


We can see that the model has an accuracy for 83% but let's take a deeper look at the model's performance.

In [20]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

        Java       0.92      0.76      0.83        45
  JavaScript       0.75      0.89      0.81        44
         PHP       0.86      0.86      0.86        35
      Python       0.83      0.83      0.83        36

    accuracy                           0.83       160
   macro avg       0.84      0.83      0.83       160
weighted avg       0.84      0.83      0.83       160



The main thing we will pay attention to here is the F1 score since it is the weighted mean of the precision and recall. As we can see the F1 score for each of the catagories is very similiar but we can see that the model is slightly better at predicting PHP.

Next let's look at the most important words to the model based on feature importance.

In [24]:
pd.Series(dict(zip(cv.get_feature_names(), tree.feature_importances_))).sort_values(ascending=False)[:6]

python        0.201688
php           0.154775
npm           0.143184
android       0.069025
java          0.061912
javascript    0.032466
dtype: float64

Above we are looking we are looking at the 6 most important words for the model. Notice that the name of each programming language that we are trying to predict appears on this list.

#### TF-IDF Model

Next we will make a model that uses how often the word appears in the document along with how many documents that word appears in, also know as TF-IDF (Term Frequency - Inverse Document Frequency). We will use the TfidfVectorizer to create a sparse matrix containing this information. We then split the data and feed the training set into a Decision Tree Classifier with a max depth of 7, which was determined through tuning on the validation set.

In [34]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.lemmatized)
y = df.language

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

tree = DecisionTreeClassifier(max_depth=7, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score: 80.62%


We can see that the model has an accuracy for 80% but let's take a deeper look at the model's performance.

In [35]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

        Java       0.94      0.69      0.79        45
  JavaScript       0.88      0.80      0.83        44
         PHP       0.91      0.83      0.87        35
      Python       0.62      0.94      0.75        36

    accuracy                           0.81       160
   macro avg       0.83      0.81      0.81       160
weighted avg       0.84      0.81      0.81       160



Again, let's mainly pay attention to the F1 score here. As we can see the F1 score for each of the catagories is less similiar than before but we can see that this model is still better at predicting PHP but is becoming worse at predicting Python.

Next let's look at the most important words to the model based on feature importance.

In [37]:
pd.Series(dict(zip(tfidf.get_feature_names(), tree.feature_importances_))).sort_values(ascending=False)[:6]

python        0.260552
php           0.189406
npm           0.152121
android       0.087964
java          0.085212
javascript    0.044568
dtype: float64

Above we are again looking we are looking at the 6 most important words for the model. Notice that the name of each programming language that we are trying to predict also appears on this list.

### Bag of Bigrams Model

Next we will make a model that uses how often a set of 2 words, also known as a bigram, appears in each document. We will use the CountVectorizer again to create a sparse matrix containing this information. We then split the data and feed the training set into a Decision Tree Classifier with a max depth of 16, which was determined through tuning on the validation set.

In [40]:
cv = CountVectorizer(ngram_range=(2, 2))
X = cv.fit_transform(df.lemmatized)
y = df.language

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

tree = DecisionTreeClassifier(max_depth=16, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score: {tree.score(X_val, y_val) * 100:.2f}%')

0.58125

We can see that the model has an accuracy for 58% but let's take a deeper look at the model's performance.

In [42]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

        Java       1.00      0.53      0.70        45
  JavaScript       0.70      0.43      0.54        44
         PHP       0.35      0.91      0.51        35
      Python       1.00      0.50      0.67        36

    accuracy                           0.58       160
   macro avg       0.76      0.59      0.60       160
weighted avg       0.78      0.58      0.60       160



Again, let's mainly pay attention to the F1 score here. As we can see the F1 score for each of the categories is very different and it does pretty poor on all of them.

Next let's look at the most important words to the model based on feature importance.

In [44]:
pd.Series(dict(zip(cv.get_feature_names(), tree.feature_importances_))).sort_values(ascending=False)[:10]

pip install                      0.218810
npm install                      0.112191
apache license                   0.103715
composer require                 0.049652
project test                     0.040399
written python                   0.035551
spring boot                      0.034574
public void                      0.032845
google cloud                     0.032004
contributingthank considering    0.030175
dtype: float64

Above we are looking we are looking at the 10 most important words for the model. Notice that we still see the word python in there but we also see some common commands like pip install and npm install.

### Bag of Trigrams Model

Lastly we will make a model that uses how often a set of 3 words, also known as a trigram, appears in each document. We will use the CountVectorizer again to create a sparse matrix containing this information. We then split the data and feed the training set into a Decision Tree Classifier with a max depth of 15, which was determined through tuning on the validation set.

In [49]:
cv = CountVectorizer(ngram_range=(3, 3))
X = cv.fit_transform(df.lemmatized)
y = df.language

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

tree = DecisionTreeClassifier(max_depth=15, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score: 43.75%


We can see that the model has an accuracy for 44% but let's take a deeper look at the model's performance.

In [50]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

        Java       1.00      0.24      0.39        45
  JavaScript       0.33      0.98      0.49        44
         PHP       1.00      0.20      0.33        35
      Python       0.90      0.25      0.39        36

    accuracy                           0.44       160
   macro avg       0.81      0.42      0.40       160
weighted avg       0.79      0.44      0.41       160



Again, let's mainly pay attention to the F1 score here. As we can see the F1 score for each of the categories is very different and it does the worst so far on all of them.

Next let's look at the most important words to the model based on feature importance.

# Best Model

So our best model was the model using the bag of words. It had both the best accuracy and the best F1 score for each category. Now let's see how it does on the test data

In [52]:
cv = CountVectorizer()
X = cv.fit_transform(df.lemmatized)
y = df.language

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

tree = DecisionTreeClassifier(max_depth=10, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score: 83.12%


In [53]:
preds = tree.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

        Java       0.79      0.71      0.75        38
  JavaScript       0.58      0.80      0.67        40
         PHP       0.91      0.81      0.85        36
      Python       0.82      0.70      0.75        46

    accuracy                           0.75       160
   macro avg       0.78      0.75      0.76       160
weighted avg       0.77      0.75      0.75       160



### Below is the code used to generate the Decision Tree Visual found in the repository; `github-prediction-tree.pdf`.

In [ ]:
# dot_data = export_graphviz(tree, 
#                            feature_names= cv.get_feature_names(),
#                            class_names= ['Java', 'JavaScript', 'PHP', 'Python'],
#                            out_file=None) 
# graph = graphviz.Source(dot_data) 

# graph.render('github-prediction-tree', view=True)